In [12]:
import boto3
from botocore.exceptions import NoCredentialsError
session = boto3.Session(profile_name='refit-divvy')
s3 = session.client('s3')

def list_s3_buckets(s3_client):
    try:
        response = s3_client.list_buckets()
        print("Existing buckets:")
        for bucket in response['Buckets']:
            print(f'  {bucket["Name"]}')
    except NoCredentialsError:
        print('Error: No credentials found')

list_s3_buckets(s3)

Existing buckets:
  aws-glue-assets-001898544471-us-east-1
  aws-glue-assets-001898544471-us-east-2
  divvy-stream-data
  lambda-layers-for-divvy
  motionsense-stream-data
  store-streaming-data
  usecases-glue-jobs
  usecases-predictions


In [26]:
import pandas as pd
import boto3

# s3 = boto3.client('s3')
obj1 = s3.get_object(Bucket='usecases-glue-jobs', Key='divvy/static/landmark_clean.csv')
obj2 = s3.get_object(Bucket='usecases-glue-jobs', Key='divvy/streamed/weather_streamed.csv')
obj3 = s3.get_object(Bucket='usecases-glue-jobs', Key='divvy/static/ohe_zipcode.csv')
l = pd.read_csv(obj1['Body'])
t = pd.read_csv('final_streamed.csv', nrows=1)
w = pd.read_csv(obj2['Body'])
z = pd.read_csv(obj3['Body'])

In [20]:
w

,Unnamed: 0,time,temp,rel_humidity,dewpoint,apparent_temp,precip,rain,snow,cloudcover,windspeed
0,56528,2019-06-14 08:00:00,16.3,64,9.6,14.8,0.0,0.0,0.0,7,18.2
1,56529,2019-06-14 09:00:00,18.0,59,9.9,17.4,0.0,0.0,0.0,42,20.0
2,56530,2019-06-14 10:00:00,19.2,54,9.7,19.1,0.0,0.0,0.0,26,21.6
3,56531,2019-06-14 11:00:00,20.1,51,9.6,20.2,0.0,0.0,0.0,27,23.2
4,56532,2019-06-14 12:00:00,20.7,49,9.8,21.0,0.0,0.0,0.0,24,25.0
...,...,...,...,...,...,...,...,...,...,...,...
31829,88357,2023-01-30 13:00:00,-9.3,65,-14.7,-16.3,0.0,0.0,0.0,51,16.3
31830,88358,2023-01-30 14:00:00,-9.2,63,-15.0,-16.4,0.0,0.0,0.0,30,17.3
31831,88359,2023-01-30 15:00:00,-9.3,61,-15.5,-16.6,0.0,0.0,0.0,20,17.1
31832,88360,2023-01-30 16:00:00,-9.6,60,-16.0,-16.2,0.0,0.0,0.0,20,13.8


In [27]:
z.head(5)

,Unnamed: 0,zip,60201,60202,60208,60301,60302,60304,60601,60602,...,60649,60651,60653,60654,60657,60659,60660,60661,60696,60804
0,0,60642,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,60647,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,60640,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,60625,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,60643,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
w.head(5)

,Unnamed: 0,time,temp,rel_humidity,dewpoint,apparent_temp,precip,rain,snow,cloudcover,windspeed
0,56528,2019-06-14 08:00:00,16.3,64,9.6,14.8,0.0,0.0,0.0,7,18.2
1,56529,2019-06-14 09:00:00,18.0,59,9.9,17.4,0.0,0.0,0.0,42,20.0
2,56530,2019-06-14 10:00:00,19.2,54,9.7,19.1,0.0,0.0,0.0,26,21.6
3,56531,2019-06-14 11:00:00,20.1,51,9.6,20.2,0.0,0.0,0.0,27,23.2
4,56532,2019-06-14 12:00:00,20.7,49,9.8,21.0,0.0,0.0,0.0,24,25.0


In [29]:
x = t.merge(w, left_on="start_time", right_on="time", how = "left").merge(l, left_on = "zip", right_on="zip_code", how='left')

In [31]:
x.drop(columns = ['Unnamed: 0_x', 'Unnamed: 0_y', "Unnamed: 0"], inplace=True)

In [33]:
x.merge(z, left_on="zip_code", right_on = "zip", how = 'left').columns

Index(['start_time', 'trips', 'zip_x', 'time', 'temp', 'rel_humidity',
       'dewpoint', 'apparent_temp', 'precip', 'rain', 'snow', 'cloudcover',
       'windspeed', 'zip_code', 'landmarks', 'Unnamed: 0', 'zip_y', '60201',
       '60202', '60208', '60301', '60302', '60304', '60601', '60602', '60603',
       '60604', '60605', '60606', '60607', '60608', '60609', '60610', '60611',
       '60612', '60613', '60614', '60615', '60616', '60617', '60618', '60619',
       '60620', '60621', '60622', '60623', '60624', '60625', '60626', '60628',
       '60629', '60630', '60632', '60636', '60637', '60638', '60640', '60641',
       '60642', '60643', '60644', '60645', '60646', '60647', '60649', '60651',
       '60653', '60654', '60657', '60659', '60660', '60661', '60696', '60804'],
      dtype='object')

In [25]:
pd.get_dummies(x['zip_code'])

,60661
0,True


In [18]:
x.drop(columns = ['Unnamed: 0_x', 'Unnamed: 0_y', "Unnamed: 0"])

,start_time,trips,zip,time,temp,rel_humidity,dewpoint,apparent_temp,precip,rain,snow,cloudcover,windspeed,zip_code,landmarks
0,2013-06-27 01:00:00,1,60661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60661,2


In [44]:
obj1 = s3.get_object(Bucket='usecases-glue-jobs', Key='divvy/static/landmark_clean.csv')
obj2 = s3.get_object(Bucket='usecases-glue-jobs', Key='divvy/streamed/weather_streamed.csv')
obj3 = s3.get_object(Bucket='usecases-glue-jobs', Key='divvy/static/ohe_zipcode.csv')
landmark = pd.read_csv(obj1['Body'])
weather = pd.read_csv(obj2['Body'])
ohe_zipcode = pd.read_csv(obj3['Body'])


def join_dataframes(incoming_data, landmark, weather):
    # Change the data types of columns to be the same before performing the merge
    incoming_data['start_time'] = pd.to_datetime(incoming_data['start_time'])
    weather['time'] = pd.to_datetime(weather['time'])
    incoming_data['zip'] = incoming_data['zip'].astype(str)
    ohe_zipcode['zip'] = ohe_zipcode['zip'].astype(str)
    landmark['zip_code'] = landmark['zip_code'].astype(str)
    
    # Perform the join operation(s) based on your requirements
    # Example: merge dataframes on a common column named 'id'
    result = incoming_data.merge(weather, left_on="start_time", right_on="time", how="left").merge(landmark, left_on="zip", right_on="zip_code", how='left')
    result = result.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'])
    result = result.merge(ohe_zipcode, left_on="zip", right_on = "zip", how = 'left')
    # result = result.drop(columns = ['zip_x', 'zip_y', "zip_code", "time"])
    
    return result



    # Create an S3 client
obj4 = s3.get_object(Bucket='usecases-glue-jobs', Key='divvy/streamed/streamed.csv')


d = pd.read_csv(obj4['Body'])
joined_data = join_dataframes(d, landmark, weather)
joined_data.head(5)

,id,start_time,trips,zip,time,temp,rel_humidity,dewpoint,apparent_temp,precip,...,60649,60651,60653,60654,60657,60659,60660,60661,60696,60804
0,1332256,2019-06-14 08:00:00,61,60661,2019-06-14 08:00:00,16.3,64,9.6,14.8,0.0,...,0,0,0,0,0,0,0,1,0,0
1,1332257,2019-06-14 08:00:00,1,60202,2019-06-14 08:00:00,16.3,64,9.6,14.8,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1332258,2019-06-14 08:00:00,15,60603,2019-06-14 08:00:00,16.3,64,9.6,14.8,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1332259,2019-06-14 08:00:00,75,60657,2019-06-14 08:00:00,16.3,64,9.6,14.8,0.0,...,0,0,0,0,1,0,0,0,0,0
4,1332260,2019-06-14 08:00:00,4,60641,2019-06-14 08:00:00,16.3,64,9.6,14.8,0.0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
joined_data.columns

Index(['id', 'start_time', 'trips', 'zip', 'time', 'temp', 'rel_humidity',
       'dewpoint', 'apparent_temp', 'precip', 'rain', 'snow', 'cloudcover',
       'windspeed', 'zip_code', 'landmarks', 'Unnamed: 0', '60201', '60202',
       '60208', '60301', '60302', '60304', '60601', '60602', '60603', '60604',
       '60605', '60606', '60607', '60608', '60609', '60610', '60611', '60612',
       '60613', '60614', '60615', '60616', '60617', '60618', '60619', '60620',
       '60621', '60622', '60623', '60624', '60625', '60626', '60628', '60629',
       '60630', '60632', '60636', '60637', '60638', '60640', '60641', '60642',
       '60643', '60644', '60645', '60646', '60647', '60649', '60651', '60653',
       '60654', '60657', '60659', '60660', '60661', '60696', '60804'],
      dtype='object')

In [41]:
joined_data.loc[:, ['start_time', "zip", "60202"]]

,start_time,zip,60202
0,2019-06-14 08:00:00,60661,0
1,2019-06-14 08:00:00,60202,1
2,2019-06-14 08:00:00,60603,0
3,2019-06-14 08:00:00,60657,0
4,2019-06-14 08:00:00,60641,0
5,2019-06-14 08:00:00,60601,0
6,2019-06-14 08:00:00,60602,0
7,2019-06-14 08:00:00,60696,0
8,2019-06-14 08:00:00,60645,0
9,2019-06-14 08:00:00,60660,0
